In [216]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer

from scipy.sparse import hstack

from sklearn.linear_model import Ridge

In [223]:
def load_data(filename):
    data = pd.read_csv(filename)
    return data

def norm_data(data):
    data['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
    data['FullDescription'] =  map(str.lower, data['FullDescription'])
    data['LocationNormalized'].fillna('nan', inplace=True)
    data['ContractTime'].fillna('nan', inplace=True) 
    return data



In [224]:
data_train = load_data('salary-train.csv')
data_test = load_data('salary-test-mini.csv')

In [225]:
data_train = norm_data(data_train)
data_test = norm_data(data_test)

In [234]:
tfidf = TfidfVectorizer(min_df=5)

X_trainFD = tfidf.fit_transform(data_train['FullDescription'])
X_testFD = tfidf.transform(data_test['FullDescription'])  

v = DictVectorizer()

X_train_LocTime = v.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_LocTime = v.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))


In [240]:
X_train = hstack([X_trainFD, X_train_LocTime])
X_test = hstack([X_testFD,X_test_LocTime])

y_train, y_test = data_train['SalaryNormalized'],data_test['SalaryNormalized']

In [241]:
clf = Ridge(alpha=1)

clf.fit(X_train, y_train)
res = clf.predict(X_test)

In [245]:
print res[0]

56563.5702372


In [248]:
with open ('q1.txt', 'w') as out:
    out.write("%.2f %.2f" %(res[0], res[1]))